In [1]:
import numpy as np
import pandas as pd
import streamlit as st
from datetime import datetime, timedelta
import os
import gspread
from google.oauth2 import service_account
from gspread_dataframe import set_with_dataframe
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from configs import *
import requests
from functionssb import *

In [2]:
gs_connection = connect_to_gs(st.secrets["gcp_service_account"])

In [20]:
drinks = fetch_drinks_data(gs_connection, drinks_table, prod_google_sheet_key, ['event','drink_size', 'start_time', 'end_time'])

In [13]:
def analyze_drinks(df):
    # Calculate the sum of the drink_size column
    total_drink_size = f"{df['drink_size'].sum()/1000:.2f}"

    # Count occurrences of specific drink types
    drink_type_counts = df['drink_type'].value_counts()

    # Get counts for specific drink types
    missed_pen_count = drink_type_counts.get('missed pen', 0)
    red_card_count = drink_type_counts.get('red card', 0)
    own_goal_count = drink_type_counts.get('own goal', 0)
    nomination_count = drink_type_counts.get('nomination', 0)

    return total_drink_size, missed_pen_count, red_card_count, own_goal_count, nomination_count


In [21]:
total_drink_size, missed_pen_count, red_card_count, own_goal_count, nomination_count = analyze_drinks(drinks)


In [22]:
nomination_count

9

In [ ]:
def render_svg_summary(path, width=None, height=None, ls=None, mps=None, rcs=None, ogs=None, nms=None):
    with open(path, "r") as f:
        svg_content = f.read()

    if ls is not None:
        svg_content = svg_content.replace("@litres", str(ls))
    if mps is not None:
        svg_content = svg_content.replace("@missed_pens", str(mps))
    if rcs is not None:
        svg_content = svg_content.replace("@red_cards", str(rcs))
    if ogs is not None:
        svg_content = svg_content.replace("@own_goals", str(ogs))
    if nms is not None:
        svg_content = svg_content.replace("@nominations", str(nms))

    b64_svg = base64.b64encode(svg_content.encode('utf-8')).decode("utf-8")

    style = ""
    if width is not None:
        style += f"width: {width}%;"
    if height is not None:
        style += f"height: {height}%;"
        
    img_tag = f'<img src="data:image/svg+xml;base64,{b64_svg}" style="{style}"/>'
    
    st.write(img_tag, unsafe_allow_html=True)